llama 3.2   : Parameters size      : 3B, Input tokens : 128,000, Output tokens : ? <br>
gpt-4o-mini : Parameters size(Apx) : 8B, Input tokens : 128,000, Output tokens : 16400

In [14]:
from pyprojroot import here
import pandas as pd
import chromadb
from openai import OpenAI
from langchain.llms import Ollama

<b>Create LLAMA client</b>

In [15]:
ollama_endpoint = "http://localhost:11434"
#ollama_api_key = ""
llama_llm = Ollama(
    model="llama3.2",
    base_url=ollama_endpoint,
    #api_key=
)

<b>Create GPT client</b>

In [16]:
gpt_llm = OpenAI(api_key="")

<b>Create vector DB client</b>

In [17]:
chroma_client = chromadb.PersistentClient(path=str(here("/Users/ashokarulsamy/Python Learning/itam/data/chroma_db/chroma_itam_ada")))

<b>Create collection in vector DB</b>

In [ ]:
#Already executed
collection = chroma_client.create_collection(name="itam_hw")

<b>Load data from csv file into data frame</b>

In [10]:
file_dir = here("/Users/ashokarulsamy/Python Learning/itam/data/itam_files/itam_hw.csv")
df = pd.read_csv(file_dir, nrows=100)

In [12]:
filtered_df = df[df['Assigned_To'] == 'Ashok Arulsamy']

In [13]:
filtered_df

,Asset_ID,Asset_Name,Asset_Type,Manufacturer,Model,Purchase_Date,Warranty_Expiry,Assigned_To,Status
1,SW002,Source Software,Software,SAP,Version-82.2,2022-11-06,2027-04-17,Ashok Arulsamy,Active
12,HW013,Majority Device,Hardware,Lenovo,Model-4924,2022-04-22,2029-10-02,Ashok Arulsamy,In Use
23,HW024,Card Device,Hardware,Lenovo,Model-0757,2022-12-27,2028-05-04,Ashok Arulsamy,Retired
52,HW053,Step Device,Hardware,Asus,Model-6226,2022-10-14,2029-05-23,Ashok Arulsamy,Available


<b>Create chunk and generate vector represenattion for chunk using embedding model</b>

In [ ]:
#Already executed
docs = []
metadatas = []
ids = []
embeddings = []
for index, row in df.iterrows():
    output_str = ""
    # Treat each row as a separate chunk
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    response = gpt_llm.embeddings.create(input = [output_str], model="text-embedding-ada-002").data[0].embedding
    embeddings.append(response)
    docs.append(output_str)
    metadatas.append({"source": "itam_hw"})
    ids.append(f"id{index}")

In [ ]:
docs

In [ ]:
print(metadatas)
print(ids)

In [ ]:
embeddings[0][:10]

<b>Vector embedding - Load vector represenation in vector DB</b>

In [ ]:
#Already executed
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

In [ ]:
print("Number of vectors in vectordb:", collection.count())

<b>Vector embedding for user query</b>

In [18]:
user_query = "I'm Ashok Arulsamy, list out assets assigned to me"
#user_query = "List of retired assets"
#user_query = "list of assets for which warrenty expires in the year 2025"
#user_query = "How many hardware assets are there" # try agent as semantic search will return huge volume of data
user_query_embeddings = gpt_llm.embeddings.create(input = [user_query], model="text-embedding-ada-002").data[0].embedding

In [19]:
user_query_embeddings

[-0.016047291457653046,
 -0.017661435529589653,
 -0.014998096972703934,
 -0.043662622570991516,
 -0.033574216067790985,
 0.03144892305135727,
 -0.03588782250881195,
 0.029350535944104195,
 -0.030157608911395073,
 -0.007855505682528019,
 0.021952372044324875,
 -0.009314961731433868,
 -0.007075335364788771,
 0.006372509989887476,
 0.011460429057478905,
 0.010747515596449375,
 0.009214077144861221,
 -0.009846284054219723,
 0.015361279249191284,
 -0.017109936103224754,
 -0.03709843009710312,
 -0.0062649003230035305,
 -0.003836957039311528,
 0.02499234490096569,
 -0.005303138867020607,
 -0.027736390009522438,
 -0.004929867573082447,
 -0.012422190979123116,
 0.002500243252143264,
 -0.01897965371608734,
 0.012570153921842575,
 -0.014486950822174549,
 -0.0007087105186656117,
 -0.018132228404283524,
 -0.03247121721506119,
 0.015603400766849518,
 0.003443509340286255,
 -0.0005359465721994638,
 0.0028667887672781944,
 -0.007963115349411964,
 0.030399730429053307,
 0.006994628347456455,
 -0.005457

<b>Create vector DB instance</b>

In [20]:
vectordb = chroma_client.get_collection(name="itam_hw")
vectordb.count()

100

<b>Semantic search for user query against vector DB</b>

In [21]:
retrieved_context = vectordb.query(
    query_embeddings = user_query_embeddings,
    n_results=5 #top_k
)

retrieved_context

{'ids': [['id23', 'id1', 'id52', 'id10', 'id6']],
 'embeddings': None,
 'documents': [['Asset_ID: HW024,\nAsset_Name: Card Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-0757,\nPurchase_Date: 2022-12-27,\nWarranty_Expiry: 2028-05-04,\nAssigned_To: Ashok Arulsamy,\nStatus: Retired,\n',
   'Asset_ID: SW002,\nAsset_Name: Source Software,\nAsset_Type: Software,\nManufacturer: SAP,\nModel: Version-82.2,\nPurchase_Date: 2022-11-06,\nWarranty_Expiry: 2027-04-17,\nAssigned_To: Ashok Arulsamy,\nStatus: Active,\n',
   'Asset_ID: HW053,\nAsset_Name: Step Device,\nAsset_Type: Hardware,\nManufacturer: Asus,\nModel: Model-6226,\nPurchase_Date: 2022-10-14,\nWarranty_Expiry: 2029-05-23,\nAssigned_To: Ashok Arulsamy,\nStatus: Available,\n',
   'Asset_ID: HW011,\nAsset_Name: Student Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-9315,\nPurchase_Date: 2024-09-04,\nWarranty_Expiry: 2026-03-25,\nAssigned_To: Anna Williams,\nStatus: Available,\n',
   'Asset_ID: HW00

<b>Generate user friendly message using LLAMA</b>

In [22]:
# Create a message with roles
messages = [
    {"role": "system", "content": "You are a knowledgeable assistant that provides detailed answers."},
    {"role": "user", "content": f"User's question: {user_query}"},
    {"role": "assistant", "content": f"Retrieved content: {retrieved_context}"}
]

# Convert messages to a single string
prompt = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in messages])

# Pass messages as input
response = llama_llm(prompt)
print(response)

Hello Ashok Arulsamy, based on the retrieved content, your assigned assets are:

1. **Card Device (Hardware)**
	* Asset ID: HW024
	* Manufacturer: Lenovo
	* Model: Model-0757
	* Purchase Date: 2022-12-27
	* Warranty Expiry: 2028-05-04
	* Assigned To: You
	* Status: Retired
2. **Source Software (Software)**
	* Asset ID: SW002
	* Manufacturer: SAP
	* Model: Version-82.2
	* Purchase Date: 2022-11-06
	* Warranty Expiry: 2027-04-17
	* Assigned To: You
	* Status: Active
3. **Step Device (Hardware)**
	* Asset ID: HW053
	* Manufacturer: Asus
	* Model: Model-6226
	* Purchase Date: 2022-10-14
	* Warranty Expiry: 2029-05-23
	* Assigned To: You
	* Status: Available
4. **Student Device (Hardware)**
	* Asset ID: HW011
	* Manufacturer: Lenovo
	* Model: Model-9315
	* Purchase Date: 2024-09-04
	* Warranty Expiry: 2026-03-25
	* Assigned To: Anna Williams
	* Status: Available (Note: Not assigned to you)
5. **Former Device (Hardware)**
	* Asset ID: HW007
	* Manufacturer: Asus
	* Model: Model-2796
	* Purch

<b>Generate user friendly message using GPT</b>

In [23]:
response = gpt_llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant that provides detailed answers."},
        {"role": "user", "content": f"User's question: {user_query}"},
        {"role": "assistant", "content": f"Retrieved content: {retrieved_context}"}
    ],
    temperature=0.0,  # Controls creativity (0.0 = deterministic, 1.0 = creative)
    max_tokens=1000,   # Adjust as needed
)

print("Generated Answer:")
print(response.choices[0].message.content)

Generated Answer:
Here are the assets assigned to you, Ashok Arulsamy:

1. **Asset ID:** HW024
   - **Asset Name:** Card Device
   - **Asset Type:** Hardware
   - **Manufacturer:** Lenovo
   - **Model:** Model-0757
   - **Purchase Date:** December 27, 2022
   - **Warranty Expiry:** May 4, 2028
   - **Status:** Retired

2. **Asset ID:** SW002
   - **Asset Name:** Source Software
   - **Asset Type:** Software
   - **Manufacturer:** SAP
   - **Model:** Version-82.2
   - **Purchase Date:** November 6, 2022
   - **Warranty Expiry:** April 17, 2027
   - **Status:** Active

3. **Asset ID:** HW053
   - **Asset Name:** Step Device
   - **Asset Type:** Hardware
   - **Manufacturer:** Asus
   - **Model:** Model-6226
   - **Purchase Date:** October 14, 2022
   - **Warranty Expiry:** May 23, 2029
   - **Status:** Available

If you need further details or assistance regarding these assets, feel free to ask!
